In [ ]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info


In [ ]:
pip install ptflops

In [ ]:
pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.41.0 requir

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
def get_ensemble(num_classes):
    # Load pretrained models
    m1 = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
    m2 = models.shufflenet_v2_x1_0(weights='IMAGENET1K_V1')
    m3 = models.mobilenet_v3_large(weights='IMAGENET1K_V1')

    # Replace classifier heads
    m1.classifier[1] = nn.Linear(m1.classifier[1].in_features, num_classes)
    m2.fc = nn.Linear(m2.fc.in_features, num_classes)
    m3.classifier[3] = nn.Linear(m3.classifier[3].in_features, num_classes)

    # Define ensemble class
    class Ensemble(nn.Module):
        def __init__(self, models_list):
            super().__init__()
            self.models_list = nn.ModuleList(models_list)

        def forward(self, x):
            outputs = [m(x) for m in self.models_list]
            return sum(outputs) / len(outputs)

    return Ensemble([m1, m2, m3])


In [ ]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]


In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def validate(model, val_loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / total, accuracy

In [ ]:
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 10
results = []

for seed in [0, 1, 2, 3, 4]:
    print(f"\n🌱 Running Seed {seed}")
    set_seed(seed)

    # Initialize model, loss, optimizer
    model = get_ensemble(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # ✅ FLOPs Calculation (handles both GMac / MMac safely)
    macs, params = get_model_complexity_info(model, (3, 32, 32),
                                             as_strings=True,
                                             print_per_layer_stat=False,
                                             verbose=False)
    macs_value = macs.replace('Mac', '').strip()
    if 'G' in macs_value:
        macs_num = float(macs_value.replace('G', '')) * 2
        flops_per_forward = f"{macs_num:.2f} GFLOPs"
    elif 'M' in macs_value:
        macs_num = float(macs_value.replace('M', '')) / 1000 * 2
        flops_per_forward = f"{macs_num:.3f} GFLOPs"
    else:
        flops_per_forward = f"{macs_value} MACs"

    print(f"Seed {seed} → FLOPs per forward pass: {flops_per_forward} | Params: {params}")

    # ✅ Energy & CO₂ Tracking
    output_dir = "/content/emissions"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    tracker = EmissionsTracker(
        project_name=f"Ensemble_CIFAR10_Seed{seed}",
        output_dir=output_dir,
        measure_power_secs=1
    )
    tracker.start()
    start_time = time.time()

    # ✅ Training Loop (4 epochs)
    for epoch in range(4):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Seed {seed} | Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # ✅ Stop Emission Tracker
    emissions = tracker.stop()
    elapsed = time.time() - start_time
    results.append((seed, emissions, elapsed, flops_per_forward))

    print(f"🌍 Seed {seed} → CO₂: {emissions:.6f} kg | Time: {elapsed/60:.2f} min | FLOPs: {flops_per_forward}")


🌱 Running Seed 0


[codecarbon WARNING @ 13:36:37] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 13:36:37] [setup] RAM Tracking...
[codecarbon INFO @ 13:36:37] [setup] CPU Tracking...


Seed 0 → FLOPs per forward pass: 0.146 GFLOPs | Params: 25.67 M


[codecarbon WARNING @ 13:36:38] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 13:36:38] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 13:36:38] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 13:36:38] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:36:38] [setup] GPU Tracking...
[codecarbon INFO @ 13:36:38] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:36:38] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 13:36:38] >>> Tracker's metadata:
[codecarbon INFO @ 13:36:38]   Platform sys

Seed 0 | Epoch 1/4 | Train Loss: 1.3826 | Val Acc: 68.75%


...

Seed 4 | Epoch 2/4 | Train Loss: 0.8253 | Val Acc: 74.60%


[codecarbon INFO @ 14:12:08] Energy consumed for RAM : 0.000642 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:12:08] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:12:08] Energy consumed for All CPU : 0.002734 kWh
[codecarbon INFO @ 14:12:08] Energy consumed for all GPUs : 0.004008 kWh. Total GPU Power : 61.70747635633629 W
[codecarbon INFO @ 14:12:08] 0.007384 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:12:09] Energy consumed for RAM : 0.000645 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:12:09] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:12:09] Energy consumed for All CPU : 0.002746 kWh
[codecarbon INFO @ 14:12:09] Energy consumed for all GPUs : 0.004025 kWh. Total GPU Power : 62.581872495950584 W
[codecarbon INFO @ 14:12:09] 0.007416 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:12:10] Energy consumed for RAM : 0.000647 kWh. RAM Power : 

Seed 4 | Epoch 3/4 | Train Loss: 0.6185 | Val Acc: 78.20%


[codecarbon INFO @ 14:14:07] Energy consumed for RAM : 0.000967 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:14:07] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:14:07] Energy consumed for All CPU : 0.004117 kWh
[codecarbon INFO @ 14:14:07] Energy consumed for all GPUs : 0.006026 kWh. Total GPU Power : 61.09407561261127 W
[codecarbon INFO @ 14:14:07] 0.011110 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:14:08] Energy consumed for RAM : 0.000969 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:14:08] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:14:08] Energy consumed for All CPU : 0.004129 kWh
[codecarbon INFO @ 14:14:08] Energy consumed for all GPUs : 0.006043 kWh. Total GPU Power : 61.86576311286186 W
[codecarbon INFO @ 14:14:08] 0.011141 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:14:09] Energy consumed for RAM : 0.000972 kWh. RAM Power : 1

Seed 4 | Epoch 4/4 | Train Loss: 0.4652 | Val Acc: 79.87%
🌍 Seed 4 → CO₂: 0.000058 kg | Time: 7.88 min | FLOPs: 0.146 GFLOPs


In [ ]:
import os
import pandas as pd # Import pandas to read the emissions.csv file

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 10
results = []

for seed in [0, 1, 2, 3, 4]:
    print(f"\n🌱 Running Seed {seed}")
    set_seed(seed)

    # Initialize model, loss, optimizer
    model = get_ensemble(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # ✅ FLOPs Calculation (handles both GMac / MMac safely)
    macs, params = get_model_complexity_info(model, (3, 32, 32),
                                             as_strings=True,
                                             print_per_layer_stat=False,
                                             verbose=False)
    macs_value = macs.replace('Mac', '').strip()
    if 'G' in macs_value:
        macs_num = float(macs_value.replace('G', '')) * 2
        flops_per_forward = f"{macs_num:.2f} GFLOPs"
    elif 'M' in macs_value:
        macs_num = float(macs_value.replace('M', '')) / 1000 * 2
        flops_per_forward = f"{macs_num:.3f} GFLOPs"
    else:
        flops_per_forward = f"{macs_value} MACs"

    print(f"Seed {seed} → FLOPs per forward pass: {flops_per_forward} | Params: {params}")

    # ✅ Energy & CO₂ Tracking
    output_dir = "/content/emissions"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    tracker = EmissionsTracker(
        project_name=f"Ensemble_CIFAR10_Seed{seed}",
        output_dir=output_dir,
        measure_power_secs=1
    )
    tracker.start()
    start_time = time.time()

    # ✅ Training Loop (4 epochs)
    for epoch in range(4):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Seed {seed} | Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # ✅ Stop Emission Tracker
    emissions: float = tracker.stop()
    elapsed = time.time() - start_time
    # Retrieve energy consumption from the tracker's results
    # Assuming the last row in the emissions.csv contains the total energy consumption for the run
    try:
        emissions_df = pd.read_csv(os.path.join(output_dir, "emissions.csv"))
        energy_consumed_kWh = emissions_df['energy_consumed'].iloc[-1]
    except Exception as e:
        print(f"Error reading emissions data for seed {seed}: {e}")
        energy_consumed_kWh = None


    results.append((seed, emissions, elapsed, flops_per_forward, energy_consumed_kWh))

    print(f"🌍 Seed {seed} → CO₂: {emissions:.6f} kg | Time: {elapsed/60:.2f} min | FLOPs: {flops_per_forward} | Energy: {energy_consumed_kWh:.6f} kWh")


🌱 Running Seed 0


[codecarbon WARNING @ 14:52:16] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:52:16] [setup] RAM Tracking...
[codecarbon INFO @ 14:52:16] [setup] CPU Tracking...


Seed 0 → FLOPs per forward pass: 0.146 GFLOPs | Params: 25.67 M


[codecarbon WARNING @ 14:52:17] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 14:52:17] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 14:52:17] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 14:52:17] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 14:52:17] [setup] GPU Tracking...
[codecarbon INFO @ 14:52:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:52:17] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 14:52:17] >>> Tracker's metadata:
[codecarbon INFO @ 14:52:17]   Platform sys

Seed 0 | Epoch 1/4 | Train Loss: 1.3826 | Val Acc: 68.75%


...

Seed 4 | Epoch 2/4 | Train Loss: 0.8253 | Val Acc: 74.60%


[codecarbon INFO @ 15:27:15] Energy consumed for RAM : 0.000645 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:27:15] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 15:27:15] Energy consumed for All CPU : 0.002747 kWh
[codecarbon INFO @ 15:27:15] Energy consumed for all GPUs : 0.004025 kWh. Total GPU Power : 64.70052571171514 W
[codecarbon INFO @ 15:27:15] 0.007416 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:27:16] Energy consumed for RAM : 0.000648 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:27:16] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 15:27:16] Energy consumed for All CPU : 0.002759 kWh
[codecarbon INFO @ 15:27:16] Energy consumed for all GPUs : 0.004042 kWh. Total GPU Power : 64.11497832180815 W
[codecarbon INFO @ 15:27:16] 0.007448 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:27:17] Energy consumed for RAM : 0.000650 kWh. RAM Power : 1

Seed 4 | Epoch 3/4 | Train Loss: 0.6185 | Val Acc: 78.20%


[codecarbon INFO @ 15:29:13] Energy consumed for RAM : 0.000967 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:29:13] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 15:29:13] Energy consumed for All CPU : 0.004118 kWh
[codecarbon INFO @ 15:29:13] Energy consumed for all GPUs : 0.006031 kWh. Total GPU Power : 60.738320863572135 W
[codecarbon INFO @ 15:29:13] 0.011116 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:29:14] Energy consumed for RAM : 0.000970 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 15:29:14] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 15:29:14] Energy consumed for All CPU : 0.004130 kWh
[codecarbon INFO @ 15:29:14] Energy consumed for all GPUs : 0.006049 kWh. Total GPU Power : 64.82516677895428 W
[codecarbon INFO @ 15:29:14] 0.011148 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:29:15] Energy consumed for RAM : 0.000972 kWh. RAM Power : 

Seed 4 | Epoch 4/4 | Train Loss: 0.4652 | Val Acc: 79.87%
🌍 Seed 4 → CO₂: 0.007014 kg | Time: 7.93 min | FLOPs: 0.146 GFLOPs | Energy: 0.014898 kWh


In [ ]:
print("\n--- Results Summary ---")
for seed, emissions, elapsed, flops, energy in results:
    print(f"Seed {seed}: CO₂ = {emissions:.6f} kg, Time = {elapsed/60:.2f} min, FLOPs = {flops}, Energy = {energy:.6f} kWh")


--- Results Summary ---
Seed 0: CO₂ = 0.006863 kg, Time = 7.74 min, FLOPs = 0.146 GFLOPs, Energy = 0.014579 kWh
Seed 1: CO₂ = 0.006835 kg, Time = 7.69 min, FLOPs = 0.146 GFLOPs, Energy = 0.014518 kWh
Seed 2: CO₂ = 0.006799 kg, Time = 7.64 min, FLOPs = 0.146 GFLOPs, Energy = 0.014441 kWh
Seed 3: CO₂ = 0.006905 kg, Time = 7.78 min, FLOPs = 0.146 GFLOPs, Energy = 0.014666 kWh
Seed 4: CO₂ = 0.007014 kg, Time = 7.93 min, FLOPs = 0.146 GFLOPs, Energy = 0.014898 kWh


In [ ]:
import pandas as pd

# Convert results to a pandas DataFrame for easier calculation
results_df = pd.DataFrame(results, columns=['seed', 'emissions', 'elapsed', 'flops', 'energy'])

# Calculate mean and standard deviation
mean_emissions = results_df['emissions'].mean()
std_emissions = results_df['emissions'].std()

mean_elapsed = results_df['elapsed'].mean()
std_elapsed = results_df['elapsed'].std()

mean_energy = results_df['energy'].mean()
std_energy = results_df['energy'].std()

print("\n--- Summary Statistics ---")
print(f"CO₂ Emissions: Mean = {mean_emissions:.6f} kg, Std Dev = {std_emissions:.6f} kg")
print(f"Training Time: Mean = {mean_elapsed/60:.2f} min, Std Dev = {std_elapsed/60:.2f} min")
print(f"Energy Consumption: Mean = {mean_energy:.6f} kWh, Std Dev = {std_energy:.6f} kWh")


--- Summary Statistics ---
CO₂ Emissions: Mean = 0.006883 kg, Std Dev = 0.000083 kg
Training Time: Mean = 7.76 min, Std Dev = 0.11 min
Energy Consumption: Mean = 0.014620 kWh, Std Dev = 0.000176 kWh
